In [ ]:
#!optimum-cli export onnx --model venkycs/securityShots urlShots/
#pip install --upgrade-strategy eager install optimum[onnxruntime]
#pip install chardet

In [1]:
from optimum.pipelines import pipeline

shotSummary = pipeline("summarization",model="urlShots/", accelerator="ort")

/home/mars/anaconda3/envs/radar/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-06 06:28:28.619283047 [W:onnxruntime:, graph.cc:3543 CleanUnusedInitializersAndNodeArgs] Removing initializer '/model/decoder/Constant_2_output_0'. It is not used by any node and should be removed from the model.
2023-09-06 06:28:28.619583546 [W:onnxruntime:, graph.cc:3543 CleanUnusedInitializersAndNodeArgs] Removing initializer '/model/decoder/Constant_2_output_0'. It is not used by any node and should be removed from the model.
2023-09-06 06:28:28.646820919 [W:onnxruntime:, graph.cc:3543 CleanUnusedInitializersAndNodeArgs] Removing initializer '/model/decoder/embed_positions/Shape_1_output_0'. It is not used by any node and should be removed from the model.
2023-09-06 06:28:28.646831394 [W:onnxruntime:, graph.cc:35

In [2]:
import requests
from bs4 import BeautifulSoup
import html2text
import re

def download_and_clean(url):
    try:
        headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"}
        response = requests.get(url,headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the main body content (remove scripts, styles, etc.)
        for script in soup(["script", "style","img","a"]):
            script.extract()
        
        body_text = soup.get_text()
        
        # Use html2text to convert HTML to clean text
        h = html2text.HTML2Text()
        h.ignore_links = True  # Ignore hyperlinks
        h.ignore_images = True  # Ignore images
        h.ignore_emphasis = True  # Ignore emphasis (italics, bold)
        h.ignore_tables = True  # Ignore tables
        clean_text = h.handle(body_text)
        
        # Remove special characters, extra whitespace, and tabs
        clean_text = re.sub(r'[^\w\s]', '', clean_text)
        clean_text = re.sub(r'\s+', ' ', clean_text).strip()
        
        return clean_text
    
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None

In [8]:
print(shotSummary(download_and_clean("https://thehackernews.com/2023/09/new-python-variant-of-chaes-malware.html")))

[{'summary_text': 'Chaes, a reworked variant of a banking and logistics malware, has been discovered in the banking and transportation industries. Chaes, which first emerged in 2020, is known to target e-commerce customers in Latin America, particularly Brazil, to steal sensitive financial information. It has undergone major overhauls, from being rewritten entirely in Python, which resulted in lower detection rates by traditional defense systems, to a comprehensive redesign and an enhanced communication protocol, Morphisec said in a new technical. The component is responsible for establishing a communication channel with the command-and-control C2 server, from where it fetches additional modules that support post-compromise activity and data theft'}]
